# TOAR REST-API Demo

In [795]:
from io import StringIO
import json
import pandas as pd
import requests
import time
from pathlib import Path
import zipfile
import io
import os
import calendar

# Configuration
Insert your access token that can be created in your user profile. A registered user is required to ensure full functionality.
<br>***Note!** Your access token will be only valid for one hour after creation.*

In [988]:
accessToken = 'eyJraWQiOiI1MTQ0NTM4ODMwNDI3Mjk1NzM2MjI2Mjc4MDM1NTUxMzg3NzY1NDMxNTY4OTIyMjAiLCJ0eXAiOiJhdCtqd3QiLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiIzNTU0ZTcyYS02MjUzLTQ2YTktOWUyZS1lMWUzNGM3MGNkNTUiLCJhdWQiOiJ0b2FyX3NlcnZpY2VzIiwic2NvcGUiOiJkaXNwbGF5X25hbWUgb2ZmbGluZV9hY2Nlc3MgZW1haWwgcHJvZmlsZSBlZHVwZXJzb25fc2NvcGVkX2FmZmlsaWF0aW9uIGVkdXBlcnNvbl9lbnRpdGxlbWVudCBlZHVwZXJzb25fdW5pcXVlX2lkIGVkdXBlcnNvbl9hc3N1cmFuY2UiLCJhdXRoX3RpbWUiOjE3NjQ3ODIzODYsImlzcyI6Imh0dHBzOi8vbG9naW4uaGVsbWhvbHR6LmRlL29hdXRoMiIsImV4cCI6MTc2NDc4NjQ2OCwiaWF0IjoxNzY0NzgyNDY4LCJqdGkiOiJmMjgzODM2OC0zZmFhLTQ4ZTItYTJkOS1lNzhmNmRkM2M0OWUiLCJjbGllbnRfaWQiOiJ0b2FyX3NlcnZpY2VzIn0.tsKoNC_4wdO2LjEpiPzVwR_r3FO5u_J4WVndIR844keQL556GtjZpfphUCQe_-3p3qygYiBpbmdcm26gOcSnWfFHdzfVUDvNtHxDnz8BcNoAmOqnygkJDZ_P9Ywi4w0ZV3LQg4LGH5C8oq4iVI0N1BNDKM7eaRX7dedhQZrH9CGOOyY6zXAE4mz2yN6Zi9TrgPhYDMvxFK3m3nYAj8fNWDZGOS0rcA1xHGxKlJrVGQcpxvxcodh2d7Wl4TfmEcx4jral9didZM-h7txjvRysiRUT0USjsb9I0XIZDf-AOiD04xt8TAbq2Cari8oW1wnxbzfvg7IxpiX_mCaheU9hbQGkSw2MbIjLlkPwU_yKI1giXaHp-exqoW2pb9AK4Ufzm-GjrbKMkVSx8VY6zSIv9WzgLk8b9WgwVCPnddSa0ml08atHEu7jlmlN96f751OzkK4MHcEwKYBLjoRgc3cw5z2GSD82GngSGuwl714tgPKy2YHJncf53Elf8VgHiLwSIqtehHO6sm87MwsckyswubCUStBkxqI29nfPMl_GsPBcCxe6CZeOajgusF5wzr85nDpGPsILShh3wUJN2-RG7CzC7xW3-WGd1wkRLlNlUY9yqoGXm5eK8e6NDHPiqcvCz3NsAmqrVlXYIs2i7snVizEb6SqzN-g0dkoL3diIHyo'

In [990]:
TOAR_SERVICE_URL="https://toar-data.fz-juelich.de/api/v2/"
headers = {'AccessToken': accessToken}
TIME_SERIES_ID = 21
flags = None

In [766]:
file_name = f"toar_time_series_{TIME_SERIES_ID}.csv"
download_folder = Path("data_downloads")
download_folder.mkdir(exist_ok = True)
output_path = download_folder / file_name

# Sample Code from TOAR

In [117]:
# get data from the REST service
flags = None
timeseries_id=21
if flags:
    result=requests.get(TOAR_SERVICE_URL+f'data/timeseries/{timeseries_id}?flags={flags}&format=csv', headers=headers)
else:
    result=requests.get(TOAR_SERVICE_URL+f'data/timeseries/{timeseries_id}?format=csv', headers=headers)
timeseries_meta = json.loads("\n".join([line[1:] for line in result.text.split('\n') if line.startswith('#')]))
df = pd.read_csv(StringIO(result.text), comment='#', header=1, sep=',',names=["time","value","flags","version","timeseries_id"],parse_dates=["time"],index_col="time",date_parser=lambda x:pd.to_datetime(x,utc=True),infer_datetime_format=True)

# print, what I got
print(f"metadata: {timeseries_meta}")
print(f"data: {df}")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [119]:
# example of how to access metadata
stationmeta=timeseries_meta["station"]
variable=timeseries_meta["variable"]
data_start_date = min(df.index).strftime("%Y-%m-%d %H:%M:%S+00")
data_end_date = min(df.index).strftime("%Y-%m-%d %H:%M:%S+00")

# show stationmeta data
print(f'retrieved data from station {stationmeta["name"]} for variable {variable["name"]} with timerange {data_start_date} - {data_end_date}')
print(f'station coordinates: {stationmeta["coordinates"]}')

retrieved data from station DUCKWORTH & ORDINANCE for variable o3 with timerange 2012-01-01 04:30:00+00 - 2012-01-01 04:30:00+00
station coordinates: {'lat': 47.56806, 'lng': -52.70222, 'alt': 7.0}


# Refined Download Script

In [768]:
print(f"Begin with download of time series {TIME_SERIES_ID} ...")

start_time = time.time()

response = requests.get(
    f"{TOAR_SERVICE_URL}data/timeseries/{TIME_SERIES_ID}?format=csv",
    #headers = headers,
    timeout=(3.05, 5)
)

end_time = time.time()
elapsed_time = end_time - start_time

# Check that your access token is valid
if response.status_code in(401, 403):
    raise RuntimeError(
        "Access token is missing, invalid, or has expired."
    )

# Check that the given URL and timeseries_id is valid
if response.status_code == 404:
    raise RuntimeError(
        "Requested resource does not exist. Please check for wrong URL or invalid timeseries_id in the configuration above."
    )
    
# Checks that the TOAR database is available
if response.status_code == 503:
    raise RuntimeError(
        "Service is not avaiable. TOAR database may be temporarily offline or overloaded."
    )

print(f"Download finished in {elapsed_time:.2f} seconds.")

# Save the downloaded data on your local drive
print(f"Create folder with name {output_path} and a download file {file_name}.")

with open(output_path, "w", encoding="utf-8") as f:
    f.write(response.text)

print("Done.")


Begin with download of time series 21 ...
Download finished in 3.17 seconds.
Create folder with name data_downloads\toar_time_series_21.csv and a download file toar_time_series_21.csv.
Done.


# Explore data access and endpoint search

In [292]:
# provides all stations from Cologne, Germany(DE)
cologne = requests.get("https://toar-data.fz-juelich.de/api/v2/stationmeta/?country=DE&state=Nordrhein-Westfalen&name=Köln&limit=None", timeout=(3.05, 5))
stations_cologne = cologne.json()
print(stations_cologne)

[{'id': 37, 'codes': ['DENW053'], 'name': 'Köln-Chorweiler', 'coordinates': {'lat': 51.019345, 'lng': 6.884636, 'alt': 45.0}, 'coordinate_validation_status': 'not checked', 'country': 'Germany', 'state': 'Nordrhein-Westfalen', 'type': 'background', 'type_of_area': 'urban', 'timezone': 'Europe/Berlin', 'additional_metadata': {'rice_production': 0.0, 'station_alt_flag': '0', 'wheat_production': 0.0, 'google_resolution': '153', 'soybean_production': 0.0, 'station_google_alt': '46', 'station_reported_alt': '45', 'station_landcover_description': 'Urbanandbuilt-up:53.9%,Croplands:20.3%,Mixedforest:17.4%,Cropland/Naturalvegetationmosaic:5.7%,EvergreenNeedleleafforest:2.1%', 'station_max_population_density_5km': '31605'}, 'aux_images': [], 'aux_docs': [], 'aux_urls': [], 'globalmeta': {'mean_topography_srtm_alt_90m_year1994': 46.0, 'mean_topography_srtm_alt_1km_year1994': 46.59005145797599, 'max_topography_srtm_relative_alt_5km_year1994': 19.0, 'min_topography_srtm_relative_alt_5km_year1994': 

In [304]:
# all timeseries from station 404 with data origin 'instrument' 
resp = requests.get(
    "https://toar-data.fz-juelich.de/api/v2/timeseries/"
    "?station_id=404"
    "&data_origin=Instrument"
    "&variable_id=13",
    timeout=(3.05, 10)
)
ts = resp.json()
print(ts)

[{'id': 19987, 'label': '', 'order': 1, 'sampling_frequency': 'hourly', 'aggregation': 'mean of two values', 'data_start_date': '2012-01-01T00:00:00+00:00', 'data_end_date': '2025-12-02T17:00:00+00:00', 'data_origin': 'instrument', 'data_origin_type': 'measurement', 'provider_version': 'N/A', 'sampling_height': 2.0, 'additional_metadata': {}, 'doi': '', 'coverage': -1.0, 'station': {'id': 404, 'codes': ['DENW212'], 'name': 'Köln Turiner Straße', 'coordinates': {'lat': 50.947367, 'lng': 6.957701, 'alt': 49.0}, 'coordinate_validation_status': 'not checked', 'country': 'Germany', 'state': 'Nordrhein-Westfalen', 'type': 'traffic', 'type_of_area': 'urban', 'timezone': 'Europe/Berlin', 'additional_metadata': {'rice_production': 0.0, 'station_alt_flag': '0', 'wheat_production': 0.0, 'google_resolution': '153', 'soybean_production': 0.0, 'station_google_alt': '55', 'station_reported_alt': '49', 'station_landcover_description': 'Urbanandbuilt-up:48.3%,Mixedforest:23.8%,Croplands:17.1%,Cropland/

In [954]:
# all timeseries from cologne stations with data origin 'instrument' 
resp = requests.get(
    "https://toar-data.fz-juelich.de/api/v2/search/?name=Köln&variable_id=13&data_origin=Instrument",
    timeout=(3.05, 10)
)
ts = resp.json()
print(ts)

[{'id': 19956, 'label': '', 'order': 1, 'sampling_frequency': 'hourly', 'aggregation': 'mean of two values', 'data_start_date': '2012-01-01T00:00:00+00:00', 'data_end_date': '2025-12-02T21:00:00+00:00', 'data_origin': 'instrument', 'data_origin_type': 'measurement', 'provider_version': 'N/A', 'sampling_height': 2.0, 'additional_metadata': {}, 'doi': '', 'coverage': -1.0, 'station': {'id': 37, 'codes': ['DENW053'], 'name': 'Köln-Chorweiler', 'coordinates': {'lat': 51.019345, 'lng': 6.884636, 'alt': 45.0}, 'coordinate_validation_status': 'not checked', 'country': 'Germany', 'state': 'Nordrhein-Westfalen', 'type': 'background', 'type_of_area': 'urban', 'timezone': 'Europe/Berlin', 'additional_metadata': {'rice_production': 0.0, 'station_alt_flag': '0', 'wheat_production': 0.0, 'google_resolution': '153', 'soybean_production': 0.0, 'station_google_alt': '46', 'station_reported_alt': '45', 'station_landcover_description': 'Urbanandbuilt-up:53.9%,Croplands:20.3%,Mixedforest:17.4%,Cropland/Na

In [974]:
resp = requests.get(
    "https://toar-data.fz-juelich.de/api/v2/timeseries/?id=19956,19958,19986,19987,31457,31544,31830,31866,31869,31887",
    timeout=(3.05, 10)
)
ts = resp.json()
print(ts)

[{'id': 19956, 'label': '', 'order': 1, 'sampling_frequency': 'hourly', 'aggregation': 'mean of two values', 'data_start_date': '2012-01-01T00:00:00+00:00', 'data_end_date': '2025-12-02T21:00:00+00:00', 'data_origin': 'instrument', 'data_origin_type': 'measurement', 'provider_version': 'N/A', 'sampling_height': 2.0, 'additional_metadata': {}, 'doi': '', 'coverage': -1.0, 'station': {'id': 37, 'codes': ['DENW053'], 'name': 'Köln-Chorweiler', 'coordinates': {'lat': 51.019345, 'lng': 6.884636, 'alt': 45.0}, 'coordinate_validation_status': 'not checked', 'country': 'Germany', 'state': 'Nordrhein-Westfalen', 'type': 'background', 'type_of_area': 'urban', 'timezone': 'Europe/Berlin', 'additional_metadata': {'rice_production': 0.0, 'station_alt_flag': '0', 'wheat_production': 0.0, 'google_resolution': '153', 'soybean_production': 0.0, 'station_google_alt': '46', 'station_reported_alt': '45', 'station_landcover_description': 'Urbanandbuilt-up:53.9%,Croplands:20.3%,Mixedforest:17.4%,Cropland/Na

In [1047]:
resp = requests.get(
    "https://toar-data.fz-juelich.de/api/v2/analysis/data/timeseries/?station_id=37,38,403,404&limit=1&data_origin=Instrument&data_origin_type=Measurement&variable_id=13&flags=AllOK", 
    headers=headers,
    timeout=(3.05, 20)
)
data = resp.json()
print(data)

{'task_id': '01d132bd-0234-4765-a01d-795292a810fa', 'status': 'https://toar-analysis.fz-juelich.de/api/v2/status/01d132bd-0234-4765-a01d-795292a810fa'}


In [1037]:
resp = requests.get(
    "https://toar-data.fz-juelich.de/api/v2/timeseries/?id=19956",
    timeout=(3.05, 10)
)
ts = resp.json()
print(ts)

[{'id': 19956, 'label': '', 'order': 1, 'sampling_frequency': 'hourly', 'aggregation': 'mean of two values', 'data_start_date': '2012-01-01T00:00:00+00:00', 'data_end_date': '2025-12-02T21:00:00+00:00', 'data_origin': 'instrument', 'data_origin_type': 'measurement', 'provider_version': 'N/A', 'sampling_height': 2.0, 'additional_metadata': {}, 'doi': '', 'coverage': -1.0, 'station': {'id': 37, 'codes': ['DENW053'], 'name': 'Köln-Chorweiler', 'coordinates': {'lat': 51.019345, 'lng': 6.884636, 'alt': 45.0}, 'coordinate_validation_status': 'not checked', 'country': 'Germany', 'state': 'Nordrhein-Westfalen', 'type': 'background', 'type_of_area': 'urban', 'timezone': 'Europe/Berlin', 'additional_metadata': {'rice_production': 0.0, 'station_alt_flag': '0', 'wheat_production': 0.0, 'google_resolution': '153', 'soybean_production': 0.0, 'station_google_alt': '46', 'station_reported_alt': '45', 'station_landcover_description': 'Urbanandbuilt-up:53.9%,Croplands:20.3%,Mixedforest:17.4%,Cropland/Na

# Data download via analysis service

In [880]:
# Download the data on a specific variable from a specific station in a specific date range
resp = requests.get( 
    "https://toar-data.fz-juelich.de/api/v2/analysis/data/timeseries/?station_id=404&limit=3&variable_id=13&daterange=2010-01-01T00:00:00,2024-12-31T23:59:59", 
    headers=headers,
    timeout=(3.05, 20) 
)

week_data = resp.json() 
print(week_data)

{'task_id': '7d29135c-3362-472a-8927-2d513bbc68db', 'status': 'https://toar-analysis.fz-juelich.de/api/v2/status/7d29135c-3362-472a-8927-2d513bbc68db'}


In [882]:
# Downlaod ZIP file from our request
result_url = 'https://toar-analysis.fz-juelich.de/api/v2/status/7d29135c-3362-472a-8927-2d513bbc68db'
resp = requests.get(result_url)
zip_content = BytesIO(resp.content)

print("Data downloaded successfully.")

# Extract JSON files
all_data = []
station_id = None
variable_name = None

with zipfile.ZipFile(zip_content, "r") as z:
    json_files = [n for n in z.namelist() if n.endswith(".json")]
    print("Gefundene JSON-Dateien:", json_files)
    
    for file_name in json_files:
        with z.open(file_name) as f:
            content = f.read().decode("utf-8").strip()
            if not content:
                print(f"File {file_name} is empty, skip.")
                continue

            # 
            lines = content.splitlines()
            json_text = "\n".join([line for line in lines if not line.startswith("#")])

            try:
                obj = json.loads(json_text)

                # Set station & variable names dynamically
                if isinstance(obj, dict):
                    station_id = obj.get('station_id', station_id)
                    variable_name = obj.get('variable_name', variable_name)

                    if "data" in obj and obj["data"]:
                        all_data.extend(obj["data"])
                elif isinstance(obj, list) and obj:
                    all_data.extend(obj)
                else:
                    print(f"File {file_name} has no data, skip.")
                    
            except json.JSONDecodeError:
                print(f"File {file_name} could not be parsed.")

# Create dataframe 
df = pd.DataFrame(all_data)
if not df.empty and 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'], utc=True)  # UTC
    df.set_index('datetime', inplace=True)  # set index

print(df.head())
print("Number of rows:", len(df))

# Save as CSV files for each day
if not df.empty:
    start_date = df.index.min().normalize()
    end_date = df.index.max().normalize()
    current_day = start_date

    while current_day <= end_date:
        next_day = current_day + timedelta(days=1)
        daily_data = df[(df.index >= current_day) & (df.index < next_day)]
        
        if not daily_data.empty:
            year = str(current_day.year)
            month_name = calendar.month_name[current_day.month]
            folder_path = os.path.join(
                "TOAR Data",
                f"Station_{station_id or 'unknown'}",
                f"Variable_{variable_name or 'unknown'}",
                year,
                month_name
            )
            os.makedirs(folder_path, exist_ok=True)
            
            filename = f"{current_day.date()}.csv"
            filepath = os.path.join(folder_path, filename)
            daily_data.to_csv(filepath, encoding="utf-8")
            print(f"{filepath} gespeichert. Zeilen: {len(daily_data)}")
        
        current_day = next_day

Data downloaded successfully.
Gefundene JSON-Dateien: ['19987.json', '31544.json', '31887.json']
                             value  flags                       version  \
datetime                                                                  
2012-01-01 00:00:00+00:00  746.203      1  000001.000000.00000000000000   
2012-01-01 01:00:00+00:00  176.818      1  000001.000000.00000000000000   
2012-01-01 02:00:00+00:00   80.217      1  000001.000000.00000000000000   
2012-01-01 03:00:00+00:00   64.593      1  000001.000000.00000000000000   
2012-01-01 04:00:00+00:00   48.118      1  000001.000000.00000000000000   

                           timeseries_id  
datetime                                  
2012-01-01 00:00:00+00:00          19987  
2012-01-01 01:00:00+00:00          19987  
2012-01-01 02:00:00+00:00          19987  
2012-01-01 03:00:00+00:00          19987  
2012-01-01 04:00:00+00:00          19987  
Number of rows: 86568
TOAR Data\Station_unknown\Variable_unknown\2012\Januar

## Via time series endpoint and as csv

In [785]:
# all timeseries from station 404 with data origin 'instrument' and inheriting the variable 'particles up to 10 µm diameter'
resp = requests.get(
    "https://toar-data.fz-juelich.de/api/v2/timeseries/"
    "?station_id=404"
    "&data_origin=Instrument"
    "&variable_id=13",
    headers=headers,
    timeout=(3.05, 10)
)
ts = resp.json()
print(ts)

[{'id': 19987, 'label': '', 'order': 1, 'sampling_frequency': 'hourly', 'aggregation': 'mean of two values', 'data_start_date': '2012-01-01T00:00:00+00:00', 'data_end_date': '2025-12-02T21:00:00+00:00', 'data_origin': 'instrument', 'data_origin_type': 'measurement', 'provider_version': 'N/A', 'sampling_height': 2.0, 'additional_metadata': {}, 'doi': '', 'coverage': -1.0, 'station': {'id': 404, 'codes': ['DENW212'], 'name': 'Köln Turiner Straße', 'coordinates': {'lat': 50.947367, 'lng': 6.957701, 'alt': 49.0}, 'coordinate_validation_status': 'not checked', 'country': 'Germany', 'state': 'Nordrhein-Westfalen', 'type': 'traffic', 'type_of_area': 'urban', 'timezone': 'Europe/Berlin', 'additional_metadata': {'rice_production': 0.0, 'station_alt_flag': '0', 'wheat_production': 0.0, 'google_resolution': '153', 'soybean_production': 0.0, 'station_google_alt': '55', 'station_reported_alt': '49', 'station_landcover_description': 'Urbanandbuilt-up:48.3%,Mixedforest:23.8%,Croplands:17.1%,Cropland/

In [787]:
# single timeseries with ID '19987' from station 404 we have found before
result = requests.get(
    "https://toar-data.fz-juelich.de/api/v2/data/timeseries/19987?format=csv",
    headers=headers,
    timeout=(3.05, 10)
)

df = pd.read_csv(StringIO(result.text), comment='#', header=1, sep=',',names=["time","value","flags","version","timeseries_id"],parse_dates=["time"],index_col="time",date_parser=lambda x:pd.to_datetime(x,utc=True),infer_datetime_format=True)
# print results from download
print(f"metadata: {timeseries_meta}")
print(f"data: {df}")

C:\Users\Nils\AppData\Local\Temp\ipykernel_26296\209808010.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(StringIO(result.text), comment='#', header=1, sep=',',names=["time","value","flags","version","timeseries_id"],parse_dates=["time"],index_col="time",date_parser=lambda x:pd.to_datetime(x,utc=True),infer_datetime_format=True)
C:\Users\Nils\AppData\Local\Temp\ipykernel_26296\209808010.py:8: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(StringIO(result.text), comment='#', header=1, sep=',',names=["time","value","flags","version","timeseries_id"],parse_dates=["time"],index_col="

metadata: {'id': 21, 'label': '', 'order': 1, 'sampling_frequency': 'hourly', 'aggregation': 'mean', 'data_start_date': '2012-01-01T03:30:00+00:00', 'data_end_date': '2012-06-15T19:30:00+00:00', 'data_origin': 'instrument', 'data_origin_type': 'measurement', 'provider_version': 'N/A', 'sampling_height': 2.0, 'additional_metadata': {}, 'data_license_accepted': None, 'dataset_approved_by_provider': None, 'doi': '', 'coverage': -1.0, 'station': {'id': 20, 'codes': ['010101', '10101'], 'name': 'DUCKWORTH & ORDINANCE', 'coordinates': {'lat': 47.56806, 'lng': -52.70222, 'alt': 7.0}, 'coordinate_validation_status': 'not checked', 'country': 'Canada', 'state': 'Newfoundland and Labrador', 'type': 'unknown', 'type_of_area': 'unknown', 'timezone': 'America/St_Johns', 'additional_metadata': {'rice_production': 0.0, 'wheat_production': 0.0, 'soybean_production': 0.0}, 'roles': None, 'annotations': None, 'aux_images': [], 'aux_docs': [], 'aux_urls': [], 'globalmeta': {'mean_topography_srtm_alt_90m_

In [789]:
# Filter a wanted date range
start_date = "2024-11-01"
end_date   = "2024-11-07"
df_filtered = df.loc[start_date:end_date]

# Locally save csv file
df_filtered.to_csv("filtered_timeseries.csv", encoding="utf-8")

print("Saved filtered data. Number of rows:", len(df_filtered))
print(df_filtered.head())

Saved filtered data. Number of rows: 168
                            value  flags                       version  \
time                                                                     
2024-11-01 00:00:00+00:00  16.405      7  000000.000001.20241103220303   
2024-11-01 01:00:00+00:00  16.882      7  000000.000001.20241103220303   
2024-11-01 02:00:00+00:00  16.739      7  000000.000001.20241103220303   
2024-11-01 03:00:00+00:00  17.721      7  000000.000001.20241103220303   
2024-11-01 04:00:00+00:00  18.714      7  000000.000001.20241103220303   

                           timeseries_id  
time                                      
2024-11-01 00:00:00+00:00          19987  
2024-11-01 01:00:00+00:00          19987  
2024-11-01 02:00:00+00:00          19987  
2024-11-01 03:00:00+00:00          19987  
2024-11-01 04:00:00+00:00          19987  


#### Date arithmetic

In [791]:
# Load csv into dataframe
df = pd.read_csv(
    StringIO(result.text),
    comment='#',
    header=0,
    names=["time","value","flags","version","timeseries_id"],
    parse_dates=["time"],
    index_col="time"
)

df.index = df.index.tz_convert('UTC') if df.index.tzinfo else df.index.tz_localize('UTC')

# define range
start_date = pd.Timestamp("2024-11-01", tz="UTC")
end_date   = pd.Timestamp("2024-11-07", tz="UTC")

# Create a folder to store the csv files
output_dir = "timeseries_csvs"
os.makedirs(output_dir, exist_ok=True)

#  Iterate over each day in time range and save each as a csv file
current_day = start_date
while current_day <= end_date:
    next_day = current_day + timedelta(days=1)
    
    # Data for one day
    mask = (df.index >= current_day) & (df.index < next_day)
    daily_data = df.loc[mask]
    
    if not daily_data.empty:
        filename = f"timeseries_{current_day.date()}.csv"
        filepath = os.path.join(output_dir, filename)
        daily_data.to_csv(filepath, encoding="utf-8")
        print(f"{filename} gespeichert. Zeilen: {len(daily_data)}")
    
    current_day = next_day

timeseries_2024-11-01.csv gespeichert. Zeilen: 24
timeseries_2024-11-02.csv gespeichert. Zeilen: 24
timeseries_2024-11-03.csv gespeichert. Zeilen: 24
timeseries_2024-11-04.csv gespeichert. Zeilen: 24
timeseries_2024-11-05.csv gespeichert. Zeilen: 24
timeseries_2024-11-06.csv gespeichert. Zeilen: 24
timeseries_2024-11-07.csv gespeichert. Zeilen: 24


In [799]:
# Convert numerical month value into its written name
month_str = calendar.month_name[current_day.month]

station_id = 404
variable_name = "PM10"  # oder dynamisch aus den Metadaten

# Load csv into dataframe
df = pd.read_csv(
    StringIO(result.text),
    comment='#',
    header=0,
    names=["time","value","flags","version","timeseries_id"],
    parse_dates=["time"],
    index_col="time"
)

# --- UTC-konvertieren ---
df.index = df.index.tz_convert('UTC') if df.index.tzinfo else df.index.tz_localize('UTC')

# --- Zeitraum definieren ---
start_date = pd.Timestamp("2024-11-01", tz="UTC")
end_date   = pd.Timestamp("2024-11-07", tz="UTC")

# --- Loop über Tage ---
current_day = start_date
while current_day <= end_date:
    next_day = current_day + timedelta(days=1)
    
    # Daten für einen Tag filtern
    mask = (df.index >= current_day) & (df.index < next_day)
    daily_data = df.loc[mask]
    
    if not daily_data.empty:
        # --- Pfad nach Station/Variable/Jahr/Monat erstellen ---
        folder_path = os.path.join(
            f"Station_{station_id}",
            f"Variable_{variable_name}",
            str(current_day.year),
            month_str
        )
        os.makedirs(folder_path, exist_ok=True)
        
        # --- CSV-Datei speichern ---
        filename = f"{current_day.date()}.csv"
        filepath = os.path.join(folder_path, filename)
        daily_data.to_csv(filepath, encoding="utf-8")
        print(f"{filepath} gespeichert. Zeilen: {len(daily_data)}")
    
    current_day = next_day

Station_404\Variable_PM10\2024\November\2024-11-01.csv gespeichert. Zeilen: 24
Station_404\Variable_PM10\2024\November\2024-11-02.csv gespeichert. Zeilen: 24
Station_404\Variable_PM10\2024\November\2024-11-03.csv gespeichert. Zeilen: 24
Station_404\Variable_PM10\2024\November\2024-11-04.csv gespeichert. Zeilen: 24
Station_404\Variable_PM10\2024\November\2024-11-05.csv gespeichert. Zeilen: 24
Station_404\Variable_PM10\2024\November\2024-11-06.csv gespeichert. Zeilen: 24
Station_404\Variable_PM10\2024\November\2024-11-07.csv gespeichert. Zeilen: 24


# Download data of variable 'particles up to 10 µm diameter' for one week from a station in Cologne

In [1051]:
print(f"Begin with download ...")
start_time = time.time()

# Request the pm10 data for station 867 in cologne for one week from May 2024
resp = requests.get( 
        "https://toar-data.fz-juelich.de/api/v2/analysis/data/timeseries/?station_id=37,38,403,404&limit=1&data_origin=Instrument&data_origin_type=Measurement&variable_id=13&flags=AllOK&daterange=2023-12-01T00:00:00,2023-12-07T23:59:59", 
    headers=headers,
    timeout=(3.05, 20) 
)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Download finished in {elapsed_time:.2f} seconds.")

status_url = resp.json().get('status', None)  # None, falls key nicht existiert
print(status_url)

Begin with download ...
Download finished in 0.79 seconds.
https://toar-analysis.fz-juelich.de/api/v2/status/9cff157e-2448-4c9d-90a8-c953c4873376


In [1053]:
# --- 1) ZIP-Datei herunterladen ---
result_url = status_url
resp = requests.get(result_url)
zip_content = BytesIO(resp.content)  # direkt in-memory

print("Data downloaded successfully.")

# --- 2) JSON-Dateien aus ZIP extrahieren und Daten sammeln ---
all_data = []
station_id = None
variable_name = None

with zipfile.ZipFile(zip_content, "r") as z:
    json_files = [n for n in z.namelist() if n.endswith(".json")]
    print("Gefundene JSON-Dateien:", json_files)
    
    for file_name in json_files:
        with z.open(file_name) as f:
            content = f.read().decode("utf-8").strip()
            if not content:
                print(f"File {file_name} is empty, skip.")
                continue

            # Kommentare überspringen
            lines = content.splitlines()
            json_text = "\n".join([line for line in lines if not line.startswith("#")])

            try:
                obj = json.loads(json_text)

                # Station / Variable dynamisch aus Metadaten
                if isinstance(obj, dict):
                    station_id = obj.get('station_id', station_id)
                    variable_name = obj.get('variable_name', variable_name)

                    if "data" in obj and obj["data"]:
                        all_data.extend(obj["data"])
                elif isinstance(obj, list) and obj:
                    all_data.extend(obj)
                else:
                    print(f"File {file_name} has no data, skip.")
                    
            except json.JSONDecodeError:
                print(f"File {file_name} could not be parsed.")

# --- 3) DataFrame erstellen ---
df = pd.DataFrame(all_data)
if not df.empty and 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'], utc=True)  # direkt UTC
    df.set_index('datetime', inplace=True)  # als Index setzen

print(df.head())
print("Number of rows:", len(df))

# --- 4) Tagesweise CSV-Dateien speichern ---
if not df.empty:
    start_date = df.index.min().normalize()
    end_date = df.index.max().normalize()
    current_day = start_date

    while current_day <= end_date:
        next_day = current_day + timedelta(days=1)
        daily_data = df[(df.index >= current_day) & (df.index < next_day)]
        
        if not daily_data.empty:
            year = str(current_day.year)
            month_name = calendar.month_name[current_day.month]
            folder_path = os.path.join(
                "TOAR Data",
                f"Station_{station_id or 'unknown'}",
                f"Variable_{variable_name or 'unknown'}",
                year,
                month_name
            )
            os.makedirs(folder_path, exist_ok=True)
            
            filename = f"{current_day.date()}.csv"
            filepath = os.path.join(folder_path, filename)
            daily_data.to_csv(filepath, encoding="utf-8")
            print(f"{filepath} gespeichert. Zeilen: {len(daily_data)}")
        
        current_day = next_day

Data downloaded successfully.
Gefundene JSON-Dateien: ['19956.json']
                            value  flags                       version  \
datetime                                                                 
2023-12-01 00:00:00+00:00  16.852      1  000001.000001.00000000000000   
2023-12-01 01:00:00+00:00  11.183      1  000001.000001.00000000000000   
2023-12-01 02:00:00+00:00  24.066      1  000001.000001.00000000000000   
2023-12-01 03:00:00+00:00  18.973      1  000001.000001.00000000000000   
2023-12-01 04:00:00+00:00  15.866      1  000001.000001.00000000000000   

                           timeseries_id  
datetime                                  
2023-12-01 00:00:00+00:00          19956  
2023-12-01 01:00:00+00:00          19956  
2023-12-01 02:00:00+00:00          19956  
2023-12-01 03:00:00+00:00          19956  
2023-12-01 04:00:00+00:00          19956  
Number of rows: 166
TOAR Data\Station_unknown\Variable_unknown\2023\December\2023-12-01.csv gespeichert. Zeilen